<a href="https://colab.research.google.com/github/vadManuel/Machine-Learning-UCF/blob/master/Homework/hw1/mvasquez_hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

## Poblem 1
Find three simple functions from R^2 to R and suitable points that are:
- a local minimum
- a local maximum
- neither a minimum nor maximum, but the gradient is zero at this point.

Plot these functions using 3D plots as in the notebooks for creating density and contour plots and three dimensional plots. Indicate clearly the points. Use 2d and 3d contour plots.

In [0]:
def plot_util(func, x, y_range):
  return np.array([func(x, y) for y in np.linspace(y_range[0], y_range[1])])

def make_surface(func, x_range, y_range):
  x = np.linspace(x_range[0], x_range[1])
  z = plot_util(func, x[0], y_range)
  for i in range(1, len(x)):
    z = np.vstack((z, plot_util(func, x[i], y_range)))

  return go.Surface(z=z, showscale=False, colorscale='Viridis')

def multi_plot(plots, title=None):
  cols = 2
  rows = 2
  specs = [[{'type': 'surface'}]*cols]*rows
  fig = make_subplots(rows=rows, cols=cols, specs=specs)

  for row in range(rows):
    for col in range(cols):
      i = row*cols+col
      if i == len(plots): break
      fig.add_trace(plots[i], row=row+1, col=col+1)
  
  fig.update_traces(contours_z=dict(show=True, color='rgb(50,50,50)'))
  fig.update_layout(title=title, autosize=False, margin=dict(l=10, r=0, b=10, t=40), height=rows*150)

  fig.show()

In [0]:
def get_H(func_xx, func_yy, func_xy, x, y):
  return func_xx(x,y)*func_yy(x,y)-func_xy(x,y)**2

def get_critical_point_type(func_xx, func_yy, func_xy, x, y):
  if get_H(func_xx, func_yy, func_xy, x, y) > 0:  # H > 0 ::= max/min
    if func_xx(x, y) > 0: # func_xx(x, y) > 0 ::= min
      return 'min'
    if func_xx(x, y) < 0: # func_xx(x, y) < 0 ::= max
      return 'max'
    return 'unknown'      # func_xx(x, y) = 0 ::= unknown
  if get_H(func_xx, func_yy, func_xy, x, y) < 0:  # H < 0 ::= saddle point
    return 'saddle point'
  return 'unknown'                                # H = 0 ::= unknown

In [0]:
def f(x, y):
  return (x+3)**2+x**2
def f_x(x, y):
  return 2*(x+3)
def f_xx(x, y):
  return 2
def f_y(x, y):
  return 2*y
def f_yy(x, y):
  return 2
def f_xy(x, y):
  return 0

def g(x,y):
  return -x**2-y**2
def g_x(x,y):
  return -2*x
def g_xx(x,y):
  return -2
def g_y(x,y):
  return -2*y
def g_yy(x,y):
  return -2
def g_xy(x,y):
  return 0

def h(x,y):
  return x**2-y**2
def h_x(x,y):
  return 2*x
def h_xx(x,y):
  return 2
def h_y(x,y):
  return -2*y
def h_yy(x,y):
  return -2
def h_xy(x,y):
  return 0

def i(x,y):
  return x**3-y**3+(x*y)**2

In [0]:
plots = [make_surface(f, (-20,20), (-20,20)), make_surface(f_x, (-20,20), (-20,20)), make_surface(f_y, (-20,20), (-20,20)), make_surface(f, (-3,-3), (0,0)),
         make_surface(g, (-20,20), (-20,20)), make_surface(g_x, (-20,20), (-20,20)), make_surface(g_y, (-20,20), (-20,20)), make_surface(g, (0,0), (0,0)),
         make_surface(h, (-20,20), (-20,20)), make_surface(h_x, (-20,20), (-20,20)), make_surface(h_y, (-20,20), (-20,20)), make_surface(h, (0,0), (0,0))]

In [18]:
multi_plot(plots[:4], title='[P1] local minimum | f(x,y)=(x+3)^2+x^2')

In [8]:
print('func {0} at ({1},{2}) is ')
print(get_H(f_xx, f_yy, f_xy, -3, 0), '\tH > 0 ::= max/min')
print('\t', f_xx(-3, 0), 'f_xx > 0 min')
print(get_H(g_xx, g_yy, g_xy, 0, 0), '\tH > 0 ::= max/min')
print('\t', g_xx(0, 0), 'f_xx < 0 max')
print(get_H(h_xx, h_yy, h_xy, 0, 0), '\tH < 0 ::= saddle point')

func {0} at ({1},{2}) is 
4 	H > 0 ::= max/min
	 2 f_xx > 0 min
4 	H > 0 ::= max/min
	 -2 f_xx < 0 max
-4 	H < 0 ::= saddle point


In [9]:
fig = go.Figure(data=[make_surface(i, (-2,1), (-1,2))])
fig.update_traces(contours_z=dict(show=True))
fig.show()